In [ ]:
from lib import layout
from lib.layout import *
import json

IdP = 'sina'

with open('conf/explorer.weights.json','r') as f:
    wstr = f.read()
with open('conf/explorer.{}.conf'.format(IdP), 'r') as f:
    for line in f:
        if 'idp_keywords' in line:
            idp_keywords = line.split('=')[-1].strip().replace('||', '|')
            break
wstr = wstr.replace('{{IdP_KEYWORDS}}', idp_keywords)
layout.WEIGHTS = json.loads(wstr)

In [ ]:
# get live screen dump if needed
from os import system
system('adb shell uiautomator dump')
system('adb pull /sdcard/window_dump.xml /tmp/screen.xml')
system('adb exec-out screencap -p > /tmp/screen.png')

In [ ]:
with open('/tmp/screen.xml','r') as f:
    xml = f.read()
with open('/tmp/screen.png', 'rb') as f:
    img = f.read()

In [ ]:
page = UniquePage(xml, '', screen_img_data=img)

## Mark and join all clickable elements in the page

In [ ]:
def markall():
#     from ipdb import set_trace; set_trace()
    page._mark_all_clickable_area()
markall()

In [ ]:
boxes = page._clickable_area
print 'count:', len(boxes), '\n', boxes

### Visualize the clickable area

In [ ]:
from IPython.core.display import display
from PIL import Image as Img
from PIL import ImageDraw
from io import BytesIO
from random import randint

In [ ]:
simg = Img.open(BytesIO(img)).convert('RGB')
draw = ImageDraw.Draw(simg, 'RGBA')
for box in boxes:
    draw.rectangle(box, fill=(randint(0,255),randint(0,255),randint(0,255),70), outline=(randint(0,255),randint(0,255),randint(0,255)))
display(simg)

## Visualize elements score

In [ ]:
score_threshold = 0.11 # filter out elements below this score

all_ue = list(page.get_all_elements())
all_ue = [u for u in all_ue if u.score >= score_threshold]
sorted_ue = sorted(all_ue, key=lambda u: u.score, reverse=True)

offset = 10
drawed_boxes = []
scores = [ue.score for ue in sorted_ue]
smin = score_threshold*1.0
smax = max(0.2, max(scores)*1.0)

from matplotlib import cm
jet = cm.get_cmap('jet')

simg = Img.open(BytesIO(img)).convert('RGB')
draw = ImageDraw.Draw(simg, 'RGBA')
for ue in sorted_ue[::-1]:
    box = ElementInfo(ue._element).bounds
    if box in drawed_boxes:
        box = ((box[0][0]+offset, box[0][1]+offset), (box[1][0]+offset, box[1][1]+offset))
    drawed_boxes.append(box)
    snorm = (ue.score-smin)/(smax-smin)
    fill = [int(c*255) for c in jet(snorm)]
    fill[3] = int(snorm*200)
    fill = tuple(fill)
    draw.rectangle(box, fill=fill, outline='green')
    draw.text(box[0], "{:.3f}".format(ue.score), fill='black')
display(simg)